# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846889609358                   -0.70    4.8         
  2   -7.852322616917       -2.26       -1.53    1.0   24.1ms
  3   -7.852614702479       -3.53       -2.56    1.5   26.4ms
  4   -7.852645974109       -4.50       -2.88    2.5   33.1ms
  5   -7.852646498823       -6.28       -3.17    1.0   24.7ms
  6   -7.852646677673       -6.75       -3.99    1.0   24.5ms
  7   -7.852646686304       -8.06       -5.22    1.5   28.4ms
  8   -7.852646686722       -9.38       -5.39    3.0   36.7ms
  9   -7.852646686729      -11.14       -6.26    1.0   25.2ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846872820468                   -0.70           4.8         
  2   -7.852554248297       -2.25       -1.63   0.80    2.0    268ms
  3   -7.852639732076       -4.07       -2.73   0.80    1.0   23.7ms
  4   -7.852646509682       -5.17       -3.37   0.80    1.8   28.2ms
  5   -7.852646677333       -6.78       -4.30   0.80    1.2   25.5ms
  6   -7.852646686585       -8.03       -4.77   0.80    2.0   30.9ms
  7   -7.852646686716       -9.88       -5.61   0.80    1.2   24.8ms
  8   -7.852646686730      -10.85       -6.23   0.80    2.2   30.5ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.381197e+01     3.095717e+00
 * time: 0.06739687919616699
     1     1.580073e+00     2.139347e+00
 * time: 0.29509401321411133
     2    -1.603868e+00     2.088466e+00
 * time: 0.3200368881225586
     3    -3.397268e+00     2.187460e+00
 * time: 0.3555769920349121
     4    -4.871459e+00     1.940291e+00
 * time: 0.3912661075592041
     5    -6.450377e+00     1.371263e+00
 * time: 0.4274110794067383
     6    -7.271432e+00     6.948440e-01
 * time: 0.46318507194519043
     7    -7.619547e+00     3.543792e-01
 * time: 0.48778295516967773
     8    -7.759358e+00     1.353346e-01
 * time: 0.5129048824310303
     9    -7.799861e+00     1.577114e-01
 * time: 0.5375440120697021
    10    -7.823919e+00     6.506014e-02
 * time: 0.5623939037322998
    11    -7.839110e+00     4.534192e-02
 * time: 0.5873398780822754
    12    -7.847338e+00     4.011979e-02
 * time: 0.6128759384155273
    13    -7.850648e+00     2.702126e-02
 * time: 0.637842

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846913128373                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645944053                   -1.64         
  2   -7.852646686730       -6.13       -3.72    1.99s
  3   -7.852646686730      -13.37       -7.27    183ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 8.057746274894355e-7
|ρ_newton - ρ_scfv| = 4.8006706700075596e-8
|ρ_newton - ρ_dm|   = 1.1517171795412542e-9
